Multihead attention 

In [36]:
import torch
import torch.nn as nn 

In [48]:
inputs = torch.nn.Embedding(4,8)

In [49]:
inputs = inputs.weight.data

In [50]:
inputs.shape

torch.Size([4, 8])

In [51]:
#1. Creating a batch of inputs
batches = torch.stack((inputs,inputs), dim = 0)

In [52]:
# output dimension
d_out = 12
d_in = 4
num_heads = 4
if d_out % num_heads != 0:
    print("Error: num_heads must be divisible by d_out, try again")


b, num_tokens, d_in = batches.shape
print(batches)
print("Batches Shape:", batches.shape)

    

# make sure d_out is always divisible by num_heads, otherwise the matrix multiplication does not work 

tensor([[[ 0.1284, -1.0791, -0.1013, -0.5682,  0.0923, -1.2324,  0.4368,
           0.4258],
         [-0.3169,  0.7074, -0.4808, -0.9596, -0.9825,  0.2914,  0.4704,
          -0.0294],
         [-0.8015, -1.2289,  0.9470, -0.5018,  2.3741,  0.7924, -0.8063,
           0.4495],
         [-0.3148,  0.1020, -1.8477, -1.4406,  0.9760,  1.0051, -0.1615,
          -0.2450]],

        [[ 0.1284, -1.0791, -0.1013, -0.5682,  0.0923, -1.2324,  0.4368,
           0.4258],
         [-0.3169,  0.7074, -0.4808, -0.9596, -0.9825,  0.2914,  0.4704,
          -0.0294],
         [-0.8015, -1.2289,  0.9470, -0.5018,  2.3741,  0.7924, -0.8063,
           0.4495],
         [-0.3148,  0.1020, -1.8477, -1.4406,  0.9760,  1.0051, -0.1615,
          -0.2450]]])
Batches Shape: torch.Size([2, 4, 8])


In [ ]:
#In a sequence of cells, go step-by-step through the same calculation of
# queries and attention scores that the MultiHeadAttention class does.
# At each step, print the tensors and their shapes to see why reshaping is necessary.

In [53]:
#setup parameter
W_query = nn.Linear(d_in, d_out, bias = False)
W_key = nn.Linear(d_in, d_out, bias = False)
W_value = nn.Linear(d_in, d_out, bias = False)

print(W_query,W_key,W_value)

keys = W_key(batches)
queries = W_query(batches)
values = W_value(batches)

print("Queries:",queries,queries.shape)
print("Keys:",keys,keys.shape)
print("Values:",values, values.shape)



Linear(in_features=8, out_features=12, bias=False) Linear(in_features=8, out_features=12, bias=False) Linear(in_features=8, out_features=12, bias=False)
Queries: tensor([[[ 0.2035, -0.5524, -0.0901, -0.0680,  0.4890,  0.2807, -0.4608,
          -0.0404,  0.0021, -0.1475, -0.2032,  0.0210],
         [-0.2377,  0.0439, -0.4297, -0.1153, -0.1148,  0.5779,  0.5144,
           0.2866, -0.0529,  0.4138, -0.0323,  0.1477],
         [ 0.7777,  0.7657,  0.8146, -1.2702, -0.0278, -0.5556, -0.8182,
          -0.1726,  0.4955, -1.4203,  0.4180,  0.4495],
         [-0.4227,  0.9470,  0.1348, -0.8359, -0.0595,  0.1494, -0.0316,
           0.8837, -0.3656, -0.4318,  0.3991,  1.1006]],

        [[ 0.2035, -0.5524, -0.0901, -0.0680,  0.4890,  0.2807, -0.4608,
          -0.0404,  0.0021, -0.1475, -0.2032,  0.0210],
         [-0.2377,  0.0439, -0.4297, -0.1153, -0.1148,  0.5779,  0.5144,
           0.2866, -0.0529,  0.4138, -0.0323,  0.1477],
         [ 0.7777,  0.7657,  0.8146, -1.2702, -0.0278, -0.5556

In [ ]:
queries = queries.reshape(b,num_tokens, num_heads, d_out // num_heads)
keys = keys.reshape(b,num_tokens, num_heads, d_out // num_heads)
values = values.reshape(b,num_tokens, num_heads, d_out // num_heads)

queries = W_query(batches)
keys = W_key(batches)
value = W_value(batches)


print("Queries reshaped:",queries,"Q shape:",queries.shape)
print("Keys reshaped:",keys,"K shape:",keys.shape)
print("Values reshaped:",values,"V shape:",values.shape) 

Queries reshaped: tensor([[[ 0.2035, -0.5524, -0.0901, -0.0680,  0.4890,  0.2807, -0.4608,
          -0.0404,  0.0021, -0.1475, -0.2032,  0.0210],
         [-0.2377,  0.0439, -0.4297, -0.1153, -0.1148,  0.5779,  0.5144,
           0.2866, -0.0529,  0.4138, -0.0323,  0.1477],
         [ 0.7777,  0.7657,  0.8146, -1.2702, -0.0278, -0.5556, -0.8182,
          -0.1726,  0.4955, -1.4203,  0.4180,  0.4495],
         [-0.4227,  0.9470,  0.1348, -0.8359, -0.0595,  0.1494, -0.0316,
           0.8837, -0.3656, -0.4318,  0.3991,  1.1006]],

        [[ 0.2035, -0.5524, -0.0901, -0.0680,  0.4890,  0.2807, -0.4608,
          -0.0404,  0.0021, -0.1475, -0.2032,  0.0210],
         [-0.2377,  0.0439, -0.4297, -0.1153, -0.1148,  0.5779,  0.5144,
           0.2866, -0.0529,  0.4138, -0.0323,  0.1477],
         [ 0.7777,  0.7657,  0.8146, -1.2702, -0.0278, -0.5556, -0.8182,
          -0.1726,  0.4955, -1.4203,  0.4180,  0.4495],
         [-0.4227,  0.9470,  0.1348, -0.8359, -0.0595,  0.1494, -0.0316,
    

In [57]:
# Transpose by head
queries = queries.transpose(1,2)
keys = keys.transpose(1,2)
values = values.transpose(1,2)

print("queries transposed:", queries)
print("queries transposed.shape:", queries.shape)
print("keys transposed:", keys)
print("keys transposed.shape:", keys.shape)
print("values transposed:", values)
print("values transposed.shape:", values.shape)


queries transposed: tensor([[[ 0.2035, -0.2377,  0.7777, -0.4227],
         [-0.5524,  0.0439,  0.7657,  0.9470],
         [-0.0901, -0.4297,  0.8146,  0.1348],
         [-0.0680, -0.1153, -1.2702, -0.8359],
         [ 0.4890, -0.1148, -0.0278, -0.0595],
         [ 0.2807,  0.5779, -0.5556,  0.1494],
         [-0.4608,  0.5144, -0.8182, -0.0316],
         [-0.0404,  0.2866, -0.1726,  0.8837],
         [ 0.0021, -0.0529,  0.4955, -0.3656],
         [-0.1475,  0.4138, -1.4203, -0.4318],
         [-0.2032, -0.0323,  0.4180,  0.3991],
         [ 0.0210,  0.1477,  0.4495,  1.1006]],

        [[ 0.2035, -0.2377,  0.7777, -0.4227],
         [-0.5524,  0.0439,  0.7657,  0.9470],
         [-0.0901, -0.4297,  0.8146,  0.1348],
         [-0.0680, -0.1153, -1.2702, -0.8359],
         [ 0.4890, -0.1148, -0.0278, -0.0595],
         [ 0.2807,  0.5779, -0.5556,  0.1494],
         [-0.4608,  0.5144, -0.8182, -0.0316],
         [-0.0404,  0.2866, -0.1726,  0.8837],
         [ 0.0021, -0.0529,  0.4955, -

In [64]:
attention_scores = queries @ keys.transpose(2,3)
print("Attention Scores:", attention_scores)
print("Attention Score shape:", attention_scores.shape)

IndexError: Dimension out of range (expected to be in range of [-3, 2], but got 3)